In [42]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

from sklearn.metrics import precision_recall_fscore_support

In [2]:
df = pd.read_csv('data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')

In [3]:
df = df.dropna()

# Throughput
df_throughput = df[['Throughput.1', 'Throughput.2', 'Throughput.3', 'Throughput.4', 'Throughput.5']].copy()
df_throughput['avg_throughput'] = df_throughput.mean(axis=1)
df_throughput['avg_throughput'] = df_throughput['avg_throughput'] / df_throughput['avg_throughput'].max()

# WorkTime
df_worktime = df[['WorkTime.1', 'WorkTime.2', 'WorkTime.3', 'WorkTime.4', 'WorkTime.5']].copy()
df_worktime['avg_worktime'] = df_worktime.mean(axis=1)
df_worktime['avg_worktime'] = df_worktime['avg_worktime'] / df_worktime['avg_worktime'].max()

In [4]:
df = df.drop(columns=['Input.msg_id', 'HITId', 'Input.sentence_id', 'Input.convo_id', 'Input.train_test_val',
                     'Input.similar_text_id', 'Input.msg_id', 'Input.timestamp', 'Input.full_text',
                      'Input.speaker', 'Input.reply_to', 
                      'Input.game_score_delta', 'Input.game_score_receiver',
                      'Input.speaker_intention', 'Input.reciever_perception', 'Input.reciever',
                      'Input.absolute_message_index', 'Input.relative_message_index', 'Input.year',
                      'Input.game_score_speaker', 'Input.num_words', 'Input.num_characters', 
                      'Input.sno', 'Input.sno1',
                      'WorkTime.1', 'WorkTime.2', 'WorkTime.3', 'WorkTime.4', 'WorkTime.5',
                      'Throughput.1', 'Throughput.2', 'Throughput.3', 'Throughput.4', 'Throughput.5',
                      'Answer.1gamemove.yes', 'Answer.2reasoning.yes',
                      'Answer.3a_apologies.yes', 'Answer.3a_compliment.yes', 'Answer.3a_personalthoughts.yes',
                      'Answer.3a_reassurance.yes', 'Answer.3rapport.yes', 'Answer.4shareinformation.yes', 'count',
                      'Answer.1gamemove.yes_pc_agree', 'Answer.2reasoning.yes_pc_agree', 
                      'Answer.3a_apologies.yes_pc_agree', 'Answer.3a_compliment.yes_pc_agree',
                      'Answer.3a_personalthoughts.yes_pc_agree', 'Answer.3a_reassurance.yes_pc_agree',
                      'Answer.3rapport.yes_pc_agree', 'Answer.4shareinformation.yes_pc_agree', 
                      'message_id.x', 'message_id.y',
                      'Answer.3a_apologies.yes_label', 'Answer.3a_compliment.yes_label',
                      'Answer.3a_personalthoughts.yes_label', 'Answer.3a_reassurance.yes_label'
                     ])

In [5]:
indices = range(df.shape[0])

In [6]:
train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].tolist()
y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_train = train.drop(columns=['Answer.1gamemove.yes_label', 'Answer.2reasoning.yes_label',
                              'Answer.3rapport.yes_label', 'Answer.4shareinformation.yes_label', 'Input.deception_quadrant'])

X_test = test.drop(columns=['Answer.1gamemove.yes_label', 'Answer.2reasoning.yes_label',
                              'Answer.3rapport.yes_label', 'Answer.4shareinformation.yes_label', 'Input.deception_quadrant'])

### Individual classifiers are KNearest Neighbours
(prev log reg)

In [43]:
# Game move classifier
clf_gamemove = KNeighborsClassifier(n_neighbors=1)
clf_gamemove.fit(X_train, y_train_gamemove)
y_pred_gamemove = clf_gamemove.predict(X_train)
y_pred_test_gamemove = clf_gamemove.predict(X_test)
print(classification_report(y_pred_gamemove, y_train_gamemove))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       651
         1.0       1.00      1.00      1.00      8441

    accuracy                           1.00      9092
   macro avg       1.00      1.00      1.00      9092
weighted avg       1.00      1.00      1.00      9092



In [44]:
# Rapport classifier
clf_rapport = KNeighborsClassifier(n_neighbors=1)
clf_rapport.fit(X_train, y_train_rapport)
y_pred_rapport = clf_rapport.predict(X_train)
y_pred_test_rapport = clf_rapport.predict(X_test)
print(classification_report(y_pred_rapport, y_train_rapport))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1262
         1.0       1.00      1.00      1.00      7830

    accuracy                           1.00      9092
   macro avg       1.00      1.00      1.00      9092
weighted avg       1.00      1.00      1.00      9092



In [45]:
# Share Information classifier
clf_shareinfo = KNeighborsClassifier(n_neighbors=1)
clf_shareinfo.fit(X_train, y_train_share_information)
y_pred_shareinfo = clf_shareinfo.predict(X_train)
y_pred_test_shareinfo = clf_shareinfo.predict(X_test)
print(classification_report(y_pred_shareinfo, y_train_share_information))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1443
         1.0       1.00      1.00      1.00      7649

    accuracy                           1.00      9092
   macro avg       1.00      1.00      1.00      9092
weighted avg       1.00      1.00      1.00      9092



In [46]:
# Reasoning classifier
clf_reasoning = KNeighborsClassifier(n_neighbors=1)
clf_reasoning.fit(X_train, y_train_reasoning)
y_pred_reasoning = clf_reasoning.predict(X_train)
y_pred_test_reasoning = clf_reasoning.predict(X_test)
print(classification_report(y_pred_reasoning, y_train_reasoning))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1555
         1.0       1.00      1.00      1.00      7537

    accuracy                           1.00      9092
   macro avg       1.00      1.00      1.00      9092
weighted avg       1.00      1.00      1.00      9092



In [ ]:
### One hot encodings 

In [47]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(y_pred_gamemove)):  
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = y_pred_gamemove[i]
    pred_obj_1['reasoning'] = y_pred_reasoning[i]
    pred_obj_1['shareinfo'] = y_pred_shareinfo[i]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = y_pred_rapport[i]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [48]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(y_pred_test_gamemove)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = y_pred_test_gamemove[i]
    pred_obj_1['reasoning'] = y_pred_test_reasoning[i]
    pred_obj_1['shareinfo'] = y_pred_test_shareinfo[i]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = y_pred_test_rapport[i]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [49]:
# Train weighted encodings
pred_df_full_throughput = pred_df_full.copy()
pred_df_full_worktime = pred_df_full.copy()
pred_df_throughput = pred_df.copy()
pred_df_worktime = pred_df.copy()

throughput_values = df_throughput['avg_throughput'].take(indices_train).values
pred_df_full_throughput = pred_df_full_throughput.mul(throughput_values, axis=0)
pred_df_throughput = pred_df_throughput.mul(throughput_values, axis=0)

worktime_values = df_worktime['avg_worktime'].take(indices_train).values
pred_df_full_worktime = pred_df_full_worktime.mul(worktime_values, axis=0)
pred_df_worktime = pred_df_worktime.mul(worktime_values, axis=0)

In [50]:
# Test weighted encodings
pred_df_full_throughput_test = pred_test_df_full.copy()
pred_df_full_worktime_test = pred_test_df_full.copy()
pred_df_throughput_test = pred_test_df.copy()
pred_df_worktime_test = pred_test_df.copy()

throughput_values_test = df_throughput['avg_throughput'].take(indices_test).values
pred_df_full_throughput_test = pred_df_full_throughput_test.mul(throughput_values_test, axis=0)
pred_df_throughput_test = pred_df_throughput_test.mul(throughput_values_test, axis=0)

worktime_values_test = df_worktime['avg_worktime'].take(indices_test).values
pred_df_full_worktime_test = pred_df_full_worktime_test.mul(worktime_values_test, axis=0)
pred_df_worktime_test = pred_df_worktime_test.mul(worktime_values_test, axis=0)

In [51]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

def f1(y_true, y_pred):
    classes_number=2
    result = 0.0
    for class_id in range(1, classes_number + 1):
        y_true_single_class = y_true[:,:,class_id]
        y_pred_single_class = y_pred[:,:,class_id]
        f1_single = f1_binary(y_true_single_class, y_pred_single_class)
        result += f1_single / float(classes_number)
    return result

In [52]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [53]:
y_train_deception = np.array(y_train_deception)
y_test_deception = np.array(y_test_deception)

In [54]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df_full,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 6ms/step - loss: 0.6939 - acc: 0.7949 - f1_m: 0.9750 - precision_m: 0.9524 - recall_m: 0.9992 - val_loss: 0.6933 - val_acc: 0.7973 - val_

In [20]:
full_pred = joint_full_model.predict(pred_df_full)
full_pred = np.argmax(full_pred, axis=1)

full_pred_test = joint_full_model.predict(pred_test_df_full)
full_pred_test = np.argmax(full_pred_test, axis=1)

In [21]:
precision_recall_fscore_support(y_test_deception, full_pred_test, average='macro')

(0.47294324681038274, 0.499767549976755, 0.48598553345388784, None)

In [55]:
y_train_rapport = np.array(y_train_rapport)
y_test_rapport = np.array(y_test_rapport)

print('Joint full model with one hot encoding, predicting rapport')
joint_full_model_rapport = create_joint_model(pred_df)
joint_full_model_rapport.summary()
history = joint_full_model_rapport.fit(x=pred_df, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport
Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 9ms/step - loss: 1.0889 - acc: 0.8704 - f1_m: 0.9245 - precision_m: 0.8608 - recall_m: 1.0000 - val_loss: 0.7982 - val_acc: 0.8404 - val_f1

143/143 [==============================] - 1s 4ms/step - loss: 0.6932 - acc: 0.7948 - f1_m: 0.9246 - precision_m: 0.8607 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.6790 - val_f1_m: 0.9285 - val_precision_m: 0.8670 - val_recall_m: 1.0000
Epoch 30/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6932 - acc: 0.7859 - f1_m: 0.9254 - precision_m: 0.8623 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.7058 - val_f1_m: 0.9285 - val_precision_m: 0.8670 - val_recall_m: 1.0000
Epoch 31/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6931 - acc: 0.7856 - f1_m: 0.9255 - precision_m: 0.8623 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.7058 - val_f1_m: 0.9285 - val_precision_m: 0.8670 - val_recall_m: 1.0000
Epoch 32/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6931 - acc: 0.7856 - f1_m: 0.9254 - precision_m: 0.8623 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.7058 - val_f1_m: 0.9285 - val_precision_m: 0.8670 - val_r

In [56]:
full_pred = joint_full_model_rapport.predict(pred_df)
full_pred = np.argmax(full_pred, axis=1)

full_pred_test = joint_full_model_rapport.predict(pred_test_df)
full_pred_test = np.argmax(full_pred_test, axis=1)

precision_recall_fscore_support(y_test_rapport, full_pred_test, average='macro')

(0.4996533912896215, 0.4994845049232299, 0.4931255866116114, None)

In [57]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_throughput_test,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_37 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_38 (Dense)             (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 6ms/step - loss: 0.6931 - acc: 0.5352 - f1_m: 0.9565 - precision_m: 0.9535 - recall_m: 0.9798 - val_loss: 0.6931 

In [58]:
full_pred = joint_full_model.predict(pred_df_full)
full_pred = np.argmax(full_pred, axis=1)

full_pred_test = joint_full_model.predict(pred_test_df_full)
full_pred_test = np.argmax(full_pred_test, axis=1)

precision_recall_fscore_support(y_test_deception, full_pred_test, average='macro')

(0.4908659448693773, 0.49185291016090077, 0.49129142564954775, None)

In [59]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model_rapport = create_joint_model(pred_df_throughput)
joint_full_model_rapport.summary()
history = joint_full_model_rapport.fit(x=pred_df_throughput, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_throughput_test,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_39 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_40 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_41 (Dense)             (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 6ms/step - loss: 0.6936 - acc: 0.1491 - f1_m: 0.9200 - precision_m: 0.8610 - recall_m: 0.9935 - val_loss: 0.6932 - 

In [60]:
full_pred_test = joint_full_model_rapport.predict(pred_df_throughput_test)
full_pred_test = np.argmax(full_pred_test, axis=1)

precision_recall_fscore_support(y_test_rapport, full_pred_test, average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4335971855760774, 0.5, 0.4644371172868582, None)

In [61]:
print('Joint full model with one hot encoding, predicting deception, weighted by worktime')
joint_full_model = create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_worktime_test,y_test_deception), callbacks=[callback])

full_pred_test = joint_full_model.predict(pred_df_full_worktime_test)
full_pred_test = np.argmax(full_pred_test, axis=1)

precision_recall_fscore_support(y_test_deception, full_pred_test, average='macro')

Joint full model with one hot encoding, predicting deception, weighted by worktime
Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_43 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 6ms/step - loss: 0.6942 - acc: 0.8248 - f1_m: 0.9752 - precision_m: 0.9525 - recall_m: 0.9993 - val_loss: 0.6933 - 

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4729551451187335, 0.5, 0.48610169491525423, None)

In [62]:
print('Joint full model with one hot encoding, predicting rapport, weighted by worktime')
joint_full_model = create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_worktime_test,y_test_rapport), callbacks=[callback])

full_pred_test = joint_full_model.predict(pred_df_worktime_test)
full_pred_test = np.argmax(full_pred_test, axis=1)

precision_recall_fscore_support(y_test_rapport, full_pred_test, average='macro')

Joint full model with one hot encoding, predicting rapport, weighted by worktime
Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_45 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_46 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_47 (Dense)             (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 6ms/step - loss: 0.6932 - acc: 0.1936 - f1_m: 0.9214 - precision_m: 0.8607 - recall_m: 0.9945 - val_loss: 0.6932 - va

(0.5664320281566212, 0.5002535496957403, 0.11778839577008055, None)

In [63]:
## Joint full model with Logistic Regression, predicting deception
clf_joint = KNeighborsClassifier(n_neighbors=1)
clf_joint.fit(pred_df_full, y_train_deception)
y_pred_deception = clf_joint.predict(pred_df_full)
y_pred_test_deception = clf_joint.predict(pred_test_df_full)
print(classification_report(y_pred_test_deception, y_test_deception))

precision_recall_fscore_support(y_test_deception, y_pred_test_deception, average='macro')

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.95      0.97      2274

    accuracy                           0.95      2274
   macro avg       0.50      0.47      0.49      2274
weighted avg       1.00      0.95      0.97      2274



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4729551451187335, 0.5, 0.48610169491525423, None)

In [64]:
## Joint full model with Logistic Regression, predicting rapport
clf_joint = KNeighborsClassifier(n_neighbors=1)
clf_joint.fit(pred_df, y_train_rapport)
y_pred_repport = clf_joint.predict(pred_df)
y_pred_test_rapport = clf_joint.predict(pred_test_df)
print(classification_report(y_pred_test_rapport, y_test_rapport))

precision_recall_fscore_support(y_test_rapport, y_pred_test_rapport, average='macro')

              precision    recall  f1-score   support

         0.0       0.01      0.15      0.02        26
         1.0       0.99      0.87      0.92      2248

    accuracy                           0.86      2274
   macro avg       0.50      0.51      0.47      2274
weighted avg       0.98      0.86      0.91      2274



(0.5106419381330414, 0.5010444232500033, 0.4742804300080915, None)

In [65]:
## Joint full model with Logistic Regression, predicting deception, weighted by throughput
clf_joint = KNeighborsClassifier(n_neighbors=1)
clf_joint.fit(pred_df_full_throughput, y_train_deception)
y_pred_deception = clf_joint.predict(pred_df_full_throughput)
y_pred_test_deception = clf_joint.predict(pred_df_full_throughput_test)
print(classification_report(y_pred_test_deception, y_test_deception))

precision_recall_fscore_support(y_test_deception, y_pred_test_deception, average='macro')

              precision    recall  f1-score   support

           0       0.07      0.08      0.07       105
           1       0.95      0.95      0.95      2169

    accuracy                           0.91      2274
   macro avg       0.51      0.51      0.51      2274
weighted avg       0.91      0.91      0.91      2274



(0.5115853256932095, 0.5099726729484868, 0.5105506822612086, None)

In [66]:
## Joint full model with Logistic Regression, predicting rapport, weighted by throughput
clf_joint = KNeighborsClassifier(n_neighbors=1)
clf_joint.fit(pred_df_throughput, y_train_rapport)
y_pred_rapport = clf_joint.predict(pred_df_throughput)
y_pred_test_rapport = clf_joint.predict(pred_df_throughput_test)
print(classification_report(y_pred_test_rapport, y_test_rapport))

precision_recall_fscore_support(y_test_rapport, y_pred_test_rapport, average='macro')

              precision    recall  f1-score   support

         0.0       0.15      0.18      0.16       248
         1.0       0.90      0.87      0.88      2026

    accuracy                           0.80      2274
   macro avg       0.52      0.53      0.52      2274
weighted avg       0.82      0.80      0.81      2274



(0.527300337547368, 0.5230327230229841, 0.5242894174359908, None)

In [67]:
## Joint full model with Logistic Regression, predicting rapport, weighted by worktime
clf_joint = KNeighborsClassifier(n_neighbors=1)
clf_joint.fit(pred_df_worktime, y_train_rapport)
y_pred_test_rapport = clf_joint.predict(pred_df_worktime_test)
print(classification_report(y_pred_test_rapport, y_test_rapport))

precision_recall_fscore_support(y_pred_test_rapport, y_test_rapport, average='macro')

              precision    recall  f1-score   support

         0.0       0.18      0.16      0.17       348
         1.0       0.85      0.87      0.86      1926

    accuracy                           0.76      2274
   macro avg       0.52      0.51      0.52      2274
weighted avg       0.75      0.76      0.76      2274



(0.5167695417970797, 0.5149004547570452, 0.5153490942100485, None)

In [68]:
## Joint full model with Logistic Regression, predicting deception, weighted by worktime
clf_joint = KNeighborsClassifier(n_neighbors=1)
clf_joint.fit(pred_df_full_worktime, y_train_deception)
y_pred_test = clf_joint.predict(pred_df_full_worktime_test)
print(classification_report(y_pred_test, y_test_deception))

precision_recall_fscore_support(y_pred_test, y_test_deception, average='macro')

              precision    recall  f1-score   support

           0       0.05      0.05      0.05       124
           1       0.95      0.95      0.95      2150

    accuracy                           0.90      2274
   macro avg       0.50      0.50      0.50      2274
weighted avg       0.90      0.90      0.90      2274



(0.4969611411595288, 0.49698424606151537, 0.496972269889217, None)